# Optimize hyperparameters of XGBoost for each mutation matrix
We will optimize hyperparameters of the eXtreme Gradient Boosting (XGBoost) model for each mutation matrix. The optimization will be based on the MAE performance of model over four validation seasons from 2012NH to 2013SH.

## Imports

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import random
from ast import literal_eval
import pickle

# self defined functions
import utilities

# for model development
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

# for hyperparameter optimization
from hyperopt import Trials, tpe, hp, fmin, space_eval
from hyperopt.pyll import scope

# for reproduciblility, fix the randomly generated numbers
SEED = 100
random.seed(SEED)
np.random.seed(SEED)

## Variables

In [ ]:
Valid_Seasons = ['2012NH', '2012SH', '2013NH', '2013SH'] # seasons from 2012NH to 2013SH

HA1_features  = [f"HA1_{x}" for x in range(1,329+1)]
meta_features = [
                 'virus',   # virus avidity
                 'serum',   # antiserum potency
                 'virusPassCat',
                 'serumPassCat'
                 ]   # metadata features

metadata   = 'a+p+vPC+sPC'   # label to record which metadata is being used
model_name = 'XGBoost'   # identifier for the type of model to be used

## Paths and filenames

In [ ]:
# paths
path_data   = "../data/"   # path of data
path_result = "../results/SuppFig6_comparison/"   # results will be saved in this directory
Path(path_result+f"/hyperopt_trials_{model_name}/").mkdir(parents=True, exist_ok=True)   # make directory if it does not exist already

# filenames
mut_mat_fn  = path_data + "aaIndID_selected.txt"   # filename of list of valid mutation matrics
optimize_fn = path_result+f"SuppFig6_optimize_{model_name}_mut_mat_hyperopt.csv"   # to save optimization results

## Read valid mutation matrices used for encoding genetic difference

In [ ]:
mut_mat_List = ['GIAG010101', 'AZAE970101', 'KOSJ950115', 'KOSJ950112', 'KOSJ950102',
                'WEIL970102', 'MIYT790101', 'MUET010101', 'RUSR970101']

# remaining
mut_mat_List = ['KOSJ950115', 'KOSJ950112', 'KOSJ950102']

## Indices of training and validation datasets for validation seasons

In [ ]:
# read dataset temporarily
dummy = pd.read_csv(path_data+"nhts_ha1_binary.csv",
                    converters={"seq_diff": literal_eval})

# to collect train and valid indices for each validation season
indices_folds = []

# loop through each validation season
for valid_season in Valid_Seasons:
    '''
    Train Test Split
        - based on seasonal framework
        - Train: past virus isolates paired with past sera
        - Test: circulating virus isolates paired with past sera
    '''
    ind_train, ind_valid = utilities.seasonal_trainTestSplit(dummy.copy(), valid_season)
    
    indices_folds.append((ind_train, ind_valid))

del dummy, ind_train, ind_valid

## Objective function for hyperopt
The objective is to minimize the average MAE over validation seasons. This function will train the RF model with provided hyperparameters and return the average MAE.

> **Parameters**
> - params (dict): dictionary of hyperparameters and corresponding values

> **Returns**
> - avg_mae (float): MAE averaged over validation seasons

In [ ]:
def objective(params):
    actual_all  = []   # to collect measured NHTs across validation seasons
    predict_all = []   # to collect predicted NHTs across validation seasons
    
    # loop through validation seasons
    for ind_train, ind_valid in indices_folds:
        '''
        Assign training and validation datasets
        '''
        # training dataset
        data_train = data.iloc[ind_train].copy()
        data_train.reset_index(drop=True, inplace=True)

        # validation dataset
        data_valid = data.iloc[ind_valid].copy()
        data_valid.reset_index(drop=True, inplace=True)
        
        
        '''
        Input features (genetic difference)
        '''
        # training dataset
        X_train = pd.DataFrame(data_train.seq_diff.to_list(),
                               index=data_train.index,
                               columns=HA1_features)
        X_train.fillna(0, inplace=True)   # replace nan with 0

        # validation dataset
        X_valid = pd.DataFrame(data_valid.seq_diff.to_list(),
                               index=data_valid.index,
                               columns=HA1_features)
        X_valid.fillna(0, inplace=True)   # replace nan with 0


        '''
        Input features (metadata features)
        '''
        X_train_meta = data_train[meta_features].fillna('None').astype('str')
        X_valid_meta = data_valid[meta_features].fillna('None').astype('str')


        # one hot encoding
        ohe = OneHotEncoder(handle_unknown='ignore')
        X_train_meta = ohe.fit_transform(X_train_meta).toarray()
        X_valid_meta = ohe.transform(X_valid_meta).toarray()

        X_train = np.hstack((X_train.values, X_train_meta))
        X_valid = np.hstack((X_valid.values, X_valid_meta))


        del X_train_meta, X_valid_meta
        
        
        '''
        Training and validation
        '''
        model = XGBRegressor(**params, booster='gbtree',
                             n_jobs=-1, random_state = SEED)
        model.fit(X_train, data_train.nht.values, verbose=False)
        predict_valid = model.predict(X_valid)
        
        '''
        save actuals and predictions
        '''
        actual_all.append(data_valid.nht.values)
        predict_all.append(predict_valid)
        
        ##################
        # End seasons loop
        ##################
    
    actuals     = np.concatenate(actual_all)
    predictions = np.concatenate(predict_all)
    
    
    '''
    metric or loss (MAE)
    '''
    avg_mae = mean_absolute_error(actuals, predictions)
    
    return avg_mae

## Optimization

In [ ]:
'''
loop through mutation matrices
'''
for mut_mat in mut_mat_List:
    
    print("Mutation matrix: ", mut_mat)
    
    '''
    Dataset
    '''
    # Genetic difference (seq_diff) encoded as per the mutation matrix
    # Converter is used to load the genetic difference saved as a list of floats
    data = pd.read_csv(path_data+f"nhts_ha1_{mut_mat}.csv",
                       converters={"seq_diff": literal_eval})
    

    '''
    Hyper-parameter optimization
    '''
    try:
        '''
        load the trials object
        '''
        with open(path_result+f"hyperopt_trials_{model_name}/trials_{mut_mat}.hyperopt", "rb") as f:
            trial     = pickle.load(f)
            max_evals = len(trial) + 5
    except:
        # hyperopt initialize trials object
        trial = Trials()
        max_evals = 100
    
    # hyperparameters search space
    space={'n_estimators': scope.int(hp.uniform('n_estimators', 10, 500)),
           'max_depth': scope.int(hp.uniform('max_depth', 2, 100)),
           'subsample': scope.float(hp.uniform('subsample', 0.1, 1)),
           'learning_rate': scope.float(hp.uniform('learning_rate', 0.001, 1)),
           'colsample_bylevel': scope.float(hp.uniform('colsample_bylevel', 0.1, 1)),
           'colsample_bytree': scope.float(hp.uniform('colsample_bytree', 0.1, 1)),
          }
    
    # hyperopt minimization
    best = fmin(fn=objective,
                space=space,
                algo=tpe.suggest,
                max_evals=max_evals, 
                trials=trial,
                rstate=np.random.default_rng(SEED))
    
    
    '''
    Best hyperparameters
    '''
    hyperparams = {'model': model_name,
                   'metadata': metadata,
                   'mut_mat': mut_mat,
                   'mae': trial.best_trial['result']['loss']}
    hyperparams.update(space_eval(space, best))
    print(hyperparams)
    
    utilities.saveDict2CSV([hyperparams], optimize_fn)
    
    
    '''
    save the trials object
    '''
    with open(path_result+f"hyperopt_trials_{model_name}/trials_{mut_mat}.hyperopt", "wb") as f:
        pickle.dump(trial, f)